<a href="https://colab.research.google.com/github/gauravsingh1995/Python_Programs/blob/master/ad_vertly_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install Dependencies**

In [ ]:
# Install Dependencies
!pip install anthropic pillow google-generativeai typing-extensions huggingface_hub
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write

**Import Libraries**

In [ ]:
# Standard libraries
from dataclasses import dataclass
from typing import Dict, List, Optional, Union, TypedDict
import json
import logging
import os

# For image handling
from PIL import Image

# For Gemini and Claude APIs
import google.generativeai as genai
from anthropic import Anthropic

# For image generation using Hugging Face InferenceClient
from huggingface_hub import InferenceClient
#For api keys
from google.colab import userdata

**Main Code**

In [ ]:

# -------------------- Logging Setup --------------------
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# -------------------- Data Structures --------------------
class EmotionAnalysis(TypedDict):
    emotion: str
    intensity: float
    reasoning: str

class AnalysisResult(TypedDict):
    input_description: str
    target_audience: str
    analysis: List[EmotionAnalysis]

@dataclass
class Config:
    gemini_api_key: str
    claude_api_key: str  # Used for analysis refinement and content generation
    hf_api_key: str     # Hugging Face API key for image generation
    model_name: str = "gemini-2.0-flash-thinking-exp-01-21"
    claude_model: str = "claude-3-5-sonnet-20241022"
    max_tokens: int = 1000

# -------------------- Helper Functions --------------------
def create_prompt(input_type: str) -> str:
    """Create the analysis prompt based on input type."""
    return f"""Identify all the emotional triggers in the {input_type} strictly from the target audience perspective.
{input_type}_description - A detailed description of the {input_type}
"target_audience" - What is the target segment of the {input_type}
"emotion" -  An easy to understood name of the emotional trigger in one word
"intensity" - A numeric value from 0-1 showing the intensity of the emotional trigger
"reasoning" - A small description of why this emotional trigger is present in the {input_type} & why it has the given intensity

Output should be in this JSON schema:
{{
    {input_type}_description: string,
    "target_audience": string,
    "analysis": [
        {{
            "emotion": string,
            "intensity": float,
            "reasoning": string
        }}
    ]
}}"""

# -------------------- Emotional Analysis & Refinement --------------------
class EmotionalAnalysisRefiner:
    def __init__(self, api_key: str, model: str):
        self.client = Anthropic(api_key=api_key)
        self.model = model

    @staticmethod
    def _calculate_metrics(analysis: List[EmotionAnalysis]) -> Dict[str, Union[float, int]]:
        if not analysis:
            return {"emotion_intensity": 0.0, "emotion_density": 0}
        emotion_intensity = sum(emotion['intensity'] for emotion in analysis) / len(analysis)
        return {
            "emotion_intensity": round(emotion_intensity, 2),
            "emotion_density": len(analysis)
        }

    def _extract_json(self, response_text: str) -> Optional[Union[Dict, List]]:
        try:
            text = response_text.strip()
            if text.startswith("```json"):
                text = text[7:]
            if text.endswith("```"):
                text = text[:-3]
            text = text.strip()
            if text.startswith('['):
                end_idx = text.rfind(']') + 1
                json_str = text[:end_idx]
            elif text.startswith('{'):
                end_idx = text.rfind('}') + 1
                json_str = text[:end_idx]
            else:
                start_idx = min([text.find(x) for x in ['{', '['] if text.find(x) != -1])
                if text[start_idx] == '[':
                    end_idx = text.rfind(']') + 1
                else:
                    end_idx = text.rfind('}') + 1
                json_str = text[start_idx:end_idx]
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            logger.error(f"Failed to parse JSON: {e}")
        return None

    def refine_analysis(
        self,
        input_content: str,
        input_description: str,
        target_audience: str,
        initial_analysis: Dict
    ) -> Dict:
        prompt = f"""Input Content: {input_content}
Input Description: {input_description}
Target Audience: {target_audience}
Initial Analysis: {json.dumps(initial_analysis)}
Analyze the "Initial Analysis" which tries to capture key emotional triggers in the "Input Content" strictly from the "Target Audience" perspective.
Verify if all emotional triggers are covered, if intensities are properly weighted, and if the reasoning is logical. Add any missing emotions in the output but do not remove any existing ones. Avoid overlapping emotions.
Return only valid JSON matching the input format."""
        try:
            response = self.client.messages.create(
                model=self.model,
                max_tokens=1000,
                messages=[{"role": "user", "content": prompt}]
            )
            print("\n" + "="*50)
            print("CLAUDE RAW RESPONSE:")
            print("="*50)
            print(response.content)
            refined_analysis = self._extract_json(response.content[0].text)
            if not refined_analysis:
                logger.warning("Using initial analysis as fallback")
                refined_analysis = initial_analysis
            else:
                print("\n" + "="*50)
                print("PARSED CLAUDE ANALYSIS:")
                print("="*50)
                print(json.dumps(refined_analysis, indent=2))
            metrics = self._calculate_metrics(refined_analysis['analysis'])
            final_output = {
                "input_description": input_description,
                "target_audience": target_audience,
                "analysis": refined_analysis['analysis'],
                "metrics": metrics
            }
            return final_output
        except Exception as e:
            logger.error(f"Error in refinement: {str(e)}", exc_info=True)
            raise

# -------------------- Content Generation --------------------
class ContentGenerator:
    def __init__(self, api_key: str, model: str):
        self.client = Anthropic(api_key=api_key)
        self.model = model

    def _extract_json(self, response_text: str) -> Optional[Union[Dict, List]]:
        try:
            text = response_text.strip()
            if text.startswith("```json"):
                text = text[7:]
            if text.endswith("```"):
                text = text[:-3]
            text = text.strip()
            if text.startswith('['):
                end_idx = text.rfind(']') + 1
                json_str = text[:end_idx]
            elif text.startswith('{'):
                end_idx = text.rfind('}') + 1
                json_str = text[:end_idx]
            else:
                start_idx = min([text.find(x) for x in ['{', '['] if text.find(x) != -1])
                if text[start_idx] == '[':
                    end_idx = text.rfind(']') + 1
                else:
                    end_idx = text.rfind('}') + 1
                json_str = text[start_idx:end_idx]
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            logger.error(f"Failed to parse JSON: {e}")
        return None

    def generate_new_concepts(
        self,
        final_analysis: Dict,
        input_type: str,
        original_input: str,
        image_size: str
    ) -> Dict:
        prompt = f"""
Input Type: {input_type}
Original Input: {original_input}
Final Analysis: {json.dumps(final_analysis, indent=2)}
Definition:
  - Emotion Intensity: The average intensity (0-1) of the detected emotional triggers.
  - Emotion Density: The total count of distinct emotional triggers identified.

Given the final emotional analysis results and the original input, generate two new image ad creative ideas for the same brand and target segment with highly enhanced emotional diversity & intensity.
The creative size should be {image_size}.

For each creative, provide the following details:
  - "title": A title for the concept.
  - "summary": A brief summary of the concept.
  - "reasoning": Explaination of why this creative?
  - "image_prompt": A detailed prompt for a text-to-image diffusion model to generate an image for the creative of size {image_size}.
  - "overlay_texts": An array of objects, each representing overlay text with the following keys: "text", "font_size", "font_type", and "location". Location is measured in x & y co-ordinates of the image
  - "brand_logo": An object specifying the "size" and "location" of the brand logo.Location is measured in x & y co-ordinates of the image

Return only valid JSON as an array of two JSON objects."""
        try:
            response = self.client.messages.create(
                model=self.model,
                max_tokens=1000,
                messages=[{"role": "user", "content": prompt}]
            )
            print("\n" + "="*50)
            print("CLAUDE NEW CONCEPTS RAW RESPONSE:")
            print("="*50)
            print(response.content)
            new_concepts = self._extract_json(response.content[0].text)
            if not new_concepts:
                logger.warning("Failed to extract new concepts JSON. Returning empty dictionary.")
                new_concepts = {}
            else:
                print("\n" + "="*50)
                print("PARSED NEW CONCEPTS:")
                print("="*50)
                print(json.dumps(new_concepts, indent=2))
            return new_concepts
        except Exception as e:
            logger.error(f"Error generating new concepts: {str(e)}", exc_info=True)
            raise

# -------------------- Image Generation --------------------
class ImageGenerator:
    def __init__(self, hf_api_key: str):
        """
        Initialize the Hugging Face InferenceClient for image generation.
        Make sure you have logged in using 'huggingface-cli login' before running this code.
        """
        try:
            self.client = InferenceClient(api_key=hf_api_key)
        except Exception as e:
            raise RuntimeError(
                "Error initializing InferenceClient. "
                "Ensure you have logged in using 'huggingface-cli login' and provided a valid Hugging Face API key."
            ) from e

    def generate_image(self, prompt: str) -> Image.Image:
        try:
            # Removed the unsupported 'size' parameter.
            # Ensure that the prompt already contains the desired image size details.
            image = self.client.text_to_image(prompt, model="black-forest-labs/FLUX.1-dev")
            return image
        except Exception as e:
            raise RuntimeError(
                f"Error generating image for prompt '{prompt}': {str(e)}"
            ) from e

    def generate_images_for_concepts(self, concepts: List[Dict], image_size: str) -> List[Optional[Image.Image]]:
        generated_images = []
        for idx, concept in enumerate(concepts):
            if 'image_prompt' in concept:
                prompt = concept['image_prompt']
                logger.info(f"Generating image for concept {idx+1} with prompt: {prompt} and image size: {image_size}")
                # Call generate_image without the image_size parameter.
                image = self.generate_image(prompt)
                generated_images.append(image)
            else:
                generated_images.append(None)
        return generated_images

# -------------------- Emotional Analyzer --------------------
class EmotionalAnalyzer:
    def __init__(self, config: Config):
        self.config = config
        self.model = self._setup_gemini()
        self.refiner = EmotionalAnalysisRefiner(config.claude_api_key, config.claude_model)

    def _setup_gemini(self) -> genai.GenerativeModel:
        genai.configure(api_key=self.config.gemini_api_key)
        return genai.GenerativeModel(self.config.model_name)

    @staticmethod
    def _clean_json_response(response: str) -> str:
        response = response.strip()
        if response.startswith('```json'):
            response = response[7:]
        if response.endswith('```'):
            response = response[:-3]
        return response.strip()

    def analyze(self, input_type: str, content: Union[str, Image.Image]) -> Dict:
        try:
            prompt = create_prompt(input_type)
            initial_result = self.model.generate_content([prompt, content])
            print("\n" + "="*50)
            print("GEMINI RAW RESPONSE:")
            print("="*50)
            print(initial_result.text)
            cleaned_output = self._clean_json_response(initial_result.text)
            initial_analysis = json.loads(cleaned_output)
            print("\n" + "="*50)
            print("PARSED GEMINI ANALYSIS:")
            print("="*50)
            print(json.dumps(initial_analysis, indent=2))
            final_analysis = self.refiner.refine_analysis(
                str(content),
                initial_analysis[f'{input_type}_description'],
                initial_analysis['target_audience'],
                initial_analysis
            )
            return final_analysis
        except Exception as e:
            logger.error(f"Error in analysis: {str(e)}", exc_info=True)
            raise

# -------------------- Input Handling --------------------
class InputHandler:
    @staticmethod
    def get_input_type() -> str:
        while True:
            input_type = input("What type of input? (image/pdf): ").lower()
            if input_type in ['image', 'pdf']:
                return input_type
            print("Please enter 'image' or 'pdf'")

    @staticmethod
    def get_file_input(input_type: str) -> Union[Image.Image, str]:
        # In Colab, use google.colab.files.upload; adjust as needed for your environment.
        from google.colab import files
        if input_type == 'image':
            print("Please upload an image file")
            uploaded = files.upload()
            if uploaded:
                image_name = list(uploaded.keys())[0]
                return Image.open(image_name)
            raise ValueError("No image uploaded")
        elif input_type == 'pdf':
            print("Please upload a PDF file")
            uploaded = files.upload()
            if uploaded:
                pdf_name = list(uploaded.keys())[0]
                try:
                    import PyPDF2
                except ImportError:
                    raise ImportError("PyPDF2 is required for PDF processing. Please install it via pip.")
                with open(pdf_name, "rb") as f:
                    reader = PyPDF2.PdfReader(f)
                    text = ""
                    for page in reader.pages:
                        text += page.extract_text() + "\n"
                return text
            raise ValueError("No PDF uploaded")

    @staticmethod
    def get_image_size() -> str:
        return input("Enter desired image size (e.g., 1024x768): ").strip()

**Run Code**

In [ ]:
if __name__ == "__main__":
    # Configure your API keys
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    CLAUDE_API_KEY = userdata.get('CLAUDE_API_KEY')
    HF_API_KEY = userdata.get('HF_TOKEN')  # For InferenceClient image generation

    # Initialize configuration
    config = Config(
        gemini_api_key=GEMINI_API_KEY,
        claude_api_key=CLAUDE_API_KEY,
        hf_api_key=HF_API_KEY
    )

    # Initialize components
    analyzer = EmotionalAnalyzer(config)
    content_generator = ContentGenerator(config.claude_api_key, config.claude_model)
    image_generator = ImageGenerator(config.hf_api_key)  # Uses Hugging Face InferenceClient
    input_handler = InputHandler()

    print("="*50)
    print("EMOTIONAL ANALYSIS SYSTEM")
    print("="*50)

    input_type = input_handler.get_input_type()
    content = input_handler.get_file_input(input_type)
    image_size = input_handler.get_image_size()

    print("\n" + "="*50)
    print("ANALYSIS STARTED")
    print("="*50)

    result = analyzer.analyze(input_type, content)
    print("\n" + "="*50)
    print("FINAL ANALYSIS WITH METRICS:")
    print("="*50)
    print(json.dumps(result, indent=2))

    new_concepts = content_generator.generate_new_concepts(
        final_analysis=result,
        input_type=input_type,
        original_input=str(content),
        image_size=image_size
    )
    print("\n" + "="*50)
    print("NEW CONCEPTS GENERATED:")
    print("="*50)
    print(json.dumps(new_concepts, indent=2))

    # Regardless of the input type, generate images for the new concepts if available.
    if isinstance(new_concepts, dict) and "concepts" in new_concepts:
        concepts_list = new_concepts["concepts"]
    elif isinstance(new_concepts, list):
        concepts_list = new_concepts
    else:
        concepts_list = []

    if concepts_list:
        generated_images = image_generator.generate_images_for_concepts(concepts_list, image_size)
        for idx, img in enumerate(generated_images):
            if img is not None:
                image_path = f"generated_concept_{idx+1}.png"
                img.save(image_path)
                print(f"Image for concept {idx+1} saved to {image_path}")
            else:
                print(f"No image prompt found for concept {idx+1}.")
    else:
        print("No valid concept with an image prompt was found; skipping image generation.")

EMOTIONAL ANALYSIS SYSTEM
What type of input? (image/pdf): image
Please upload an image file


Saving puma-forever-faster-brand-campaign-mondo-duplantis.jpg to puma-forever-faster-brand-campaign-mondo-duplantis (2).jpg
Enter desired image size (e.g., 1024x768): 1024x1024

ANALYSIS STARTED

GEMINI RAW RESPONSE:
```json
{
    "image_description": "An advertisement for Puma featuring a pole vaulter in mid-air, upside down, against a blue and white gradient background with the Puma logo and the text \"SEE THE GAME LIKE WE DO FOREVER. FASTER.\"",
    "target_audience": "Athletes, sports enthusiasts, and individuals interested in fitness and performance, particularly those who value speed, agility, and pushing personal limits.",
    "analysis": [
        {
            "emotion": "Aspiration",
            "intensity": 0.8,
            "reasoning": "The image of a pole vaulter at the peak of their jump, defying gravity, is highly aspirational. It suggests pushing boundaries and achieving great heights, which resonates with athletes striving for improvement."
        },
        {
       